### Generalized Least Squares with Statsmodels

In [10]:
import statsmodels.api as sm

In [12]:
# Longley datasets time series
data = sm.datasets.longley.load()
data.exog = sm.add_constant(data.exog)

In [17]:
ols_resid = sm.OLS(data.endog, data.exog).fit().resid
ols_resid

0     267.340030
1     -94.013942
2      46.287168
3    -410.114622
4     309.714591
5    -249.311215
6    -164.048956
7     -13.180357
8      14.304773
9     455.394095
10    -17.268927
11    -39.055043
12   -155.549974
13    -85.671308
14    341.931514
15   -206.757825
dtype: float64

In [43]:
resid_fit = sm.OLS(ols_resid[1:].reset_index(drop=True), sm.add_constant(ols_resid[:-1])).fit()

In [67]:
print(resid_fit.tvalues[1:].values[0])
print(resid_fit.pvalues[1:].values[0])

-1.4390229839613828
0.17378444789154032


In [69]:
rho = resid_fit.params[1:].values[0]

In [70]:
print(rho)

-0.3634294908742479


In [56]:
from scipy.linalg import toeplitz

toeplitz(range(5))

array([[0, 1, 2, 3, 4],
       [1, 0, 1, 2, 3],
       [2, 1, 0, 1, 2],
       [3, 2, 1, 0, 1],
       [4, 3, 2, 1, 0]])

In [57]:
order = toeplitz(range(len(ols_resid)))

In [58]:
print(order)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 2  1  0  1  2  3  4  5  6  7  8  9 10 11 12 13]
 [ 3  2  1  0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 4  3  2  1  0  1  2  3  4  5  6  7  8  9 10 11]
 [ 5  4  3  2  1  0  1  2  3  4  5  6  7  8  9 10]
 [ 6  5  4  3  2  1  0  1  2  3  4  5  6  7  8  9]
 [ 7  6  5  4  3  2  1  0  1  2  3  4  5  6  7  8]
 [ 8  7  6  5  4  3  2  1  0  1  2  3  4  5  6  7]
 [ 9  8  7  6  5  4  3  2  1  0  1  2  3  4  5  6]
 [10  9  8  7  6  5  4  3  2  1  0  1  2  3  4  5]
 [11 10  9  8  7  6  5  4  3  2  1  0  1  2  3  4]
 [12 11 10  9  8  7  6  5  4  3  2  1  0  1  2  3]
 [13 12 11 10  9  8  7  6  5  4  3  2  1  0  1  2]
 [14 13 12 11 10  9  8  7  6  5  4  3  2  1  0  1]
 [15 14 13 12 11 10  9  8  7  6  5  4  3  2  1  0]]


In [71]:
sigma = rho ** order
gls_model = sm.GLS(data.endog, data.exog, sigma = sigma)

In [72]:
gls_results = gls_model.fit()

In [73]:
gls_results.summary()

/home/johnadi/.local/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            GLS Regression Results                            
==============================================================================
Dep. Variable:                 TOTEMP   R-squared:                       0.998
Model:                            GLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     724.0
Date:                Wed, 08 May 2024   Prob (F-statistic):           1.48e-11
Time:                        12:06:39   Log-Likelihood:                -107.50
No. Observations:                  16   AIC:                             229.0
Df Residuals:                       9   BIC:                             234.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.798e+06   6.71e+05     -5.663      0.000   -5.32e+06   -2.28e+06
GNPDEFL      -12.7656     69.431     -0.184      0.858    -169.829     144.298
GNP           -0.0380      0.026     -1.448      0.182      -0.097       0.021
UNEMP         -2.1869      0.382     -5.719      0.000      -3.052      -1.322
ARMED         -1.1518      0.165     -6.970      0.000      -1.526      -0.778
POP           -0.0681      0.176     -0.386      0.709      -0.467       0.331
YEAR        1993.9529    342.635      5.819      0.000    1218.860    2769.046
==============================================================================
Omnibus:                        0.117   Durbin-Watson:                   2.611
Prob(Omnibus):                  0.943   Jarque-Bera (JB):                0.340
Skew:                          -0.036   Prob(JB):                        0.844
Kurtosis:                       2.290   Cond. No.                     5.61e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.61e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [76]:
glsar_model = sm.GLSAR(data.endog, data.exog, 1)
glsar_results = glsar_model.iterative_fit(1)
glsar_results.summary()

/home/johnadi/.local/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           GLSAR Regression Results                           
==============================================================================
Dep. Variable:                 TOTEMP   R-squared:                       0.996
Model:                          GLSAR   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     295.2
Date:                Wed, 08 May 2024   Prob (F-statistic):           6.09e-09
Time:                        12:08:46   Log-Likelihood:                -102.04
No. Observations:                  15   AIC:                             218.1
Df Residuals:                       8   BIC:                             223.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.468e+06   8.72e+05     -3.979      0.004   -5.48e+06   -1.46e+06
GNPDEFL       34.5568     84.734      0.408      0.694    -160.840     229.953
GNP           -0.0343      0.033     -1.047      0.326      -0.110       0.041
UNEMP         -1.9621      0.481     -4.083      0.004      -3.070      -0.854
ARMED         -1.0020      0.211     -4.740      0.001      -1.489      -0.515
POP           -0.0978      0.225     -0.435      0.675      -0.616       0.421
YEAR        1823.1829    445.829      4.089      0.003     795.100    2851.266
==============================================================================
Omnibus:                        1.960   Durbin-Watson:                   2.554
Prob(Omnibus):                  0.375   Jarque-Bera (JB):                1.423
Skew:                           0.713   Prob(JB):                        0.491
Kurtosis:                       2.508   Cond. No.                     4.80e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.8e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [78]:
print(gls_results.params)
print(glsar_results.params)
print("==============================")
print(gls_results.bse)
print(glsar_results.bse)

const     -3.797855e+06
GNPDEFL   -1.276565e+01
GNP       -3.800132e-02
UNEMP     -2.186949e+00
ARMED     -1.151776e+00
POP       -6.805356e-02
YEAR       1.993953e+03
dtype: float64
const     -3.467961e+06
GNPDEFL    3.455678e+01
GNP       -3.434101e-02
UNEMP     -1.962144e+00
ARMED     -1.001973e+00
POP       -9.780460e-02
YEAR       1.823183e+03
dtype: float64
const      670688.699310
GNPDEFL        69.430807
GNP             0.026248
UNEMP           0.382393
ARMED           0.165253
POP             0.176428
YEAR          342.634628
dtype: float64
const      871584.051696
GNPDEFL        84.733715
GNP             0.032803
UNEMP           0.480545
ARMED           0.211384
POP             0.224774
YEAR          445.828748
dtype: float64
